# Librería FB Prophet aplicada a datos de energía consultados en InfluxDB con el Addon para Home Assistant **Prophet InfluxDB Addon**
Este documento ha sido utilizado para probar paso a paso las funciones implementadas en `main.py` del complemento para Home Assistant **Prophet InfluxDB Addon**.
Finalmente se va a comprobar el correcto funcionamiento de la implementación que se ha hecho en el contenedor Docker que será necesario tener instalado en Home Assistant para poder hacer uso de la librería Prophet y que no ha sido posible ejecutar directamente en el custom component en equipos que corren Home Assistant OS.

El resultado de este desarrollo queda finalmente expuesto en 
https://github.com/mgenrique/hassos_prophet_addon

Las pruebas se van a realizar sobre los dos sensores que nos proporcionan el consumo de la vivienda en el caso de sistemas Victron Energy con dispositivos GX:
- Consumo acin to acout: "sensor.victron_vebus_acin1toacout_228" (En influx se encuentra en `victron_vebus_acin1toacout_228`)
- Consumo inverter to ac out: "sensor.victron_vebus_invertertoacout_228" (En influx se encuentra en `victron_vebus_invertertoacout_228`)


In [ ]:

from influxdb import InfluxDBClient
import logging
import numpy as np
import pandas as pd
from prophet import Prophet

host = "192.168.0.100"
port = 8086
user = "homeassistant"
password = "xxx"
dbname = "homeassistant"


## Definir funciones que simulan el funcionamiento del Custom Component de Home Assistant
Resultará conveniente que las siguientes funciones formen parte del custom component para facilitar la creación de consultas a InfluxQL en un formato adecuado.
Estas funciones también se incluyen como parte del script `test.py` facilitado en el repositorio que sirve de ayuda para la utilización del addon.


In [18]:
import pytz
from datetime import datetime, timedelta    
class_local_timezone=pytz.timezone('Europe/Madrid')
       
def format_date_for_influxdb( date) -> str:
    """
    Return a date and time with timezone information in the format '2024-10-01T00:00:00Z' (UTC timezone).
    """
    date = pd.to_datetime(date)
    # Check if date has a localized time
    if date.tzinfo is None:
        if class_local_timezone is None:
            date_tz = 'Europe/Madrid'        
            date = date.tz_localize(date_tz)
        else:
            date = date.tz_localize(class_local_timezone)
    # Localize the date and time to the UTC timezone
    # date = await asyncio.to_thread(date.tz_convert, 'UTC')
    date = date.tz_convert('UTC')
    # Format the date in ISO 8601 format
    date = date.strftime('%Y-%m-%dT%H:%M:%SZ')
    return date

def energy_query_string(entity_id, start: str | None = None, end: str | None = None) -> str:
    """Create the SQL query string for energy data."""
    if start is not None:
        # start = await self.format_date_for_influxdb(start)
        start = format_date_for_influxdb(start)
    if end is not None:
        # end = await self.format_date_for_influxdb(end)
        end = format_date_for_influxdb(end)
    # If start and end are not specified, return all data
    if start is None and end is None:
        query = """SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE "entity_id"='{}' GROUP BY time(1h) fill(previous)""".format(entity_id)
    elif start is None:
        query = """SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time <= '{}') AND "entity_id"='{}' GROUP BY time(1h) fill(previous)""".format(end, entity_id) 
    elif end is None:
        query = """SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time >= '{}') AND "entity_id"='{}' GROUP BY time(1h) fill(previous)""".format(start, entity_id)   
    else:
        query = """SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time >= '{}') AND (time <= '{}') AND "entity_id"='{}' GROUP BY time(1h) fill(previous)""".format(start, end, entity_id)
    return query   

## Estrategía general
Dado que influxDB trabaja siempre con fechas localizadas con timezone UTC y Prophet no permite que las fechas tengan información de TimeZone, se debe plantear una estrategia para alinear correctamente las fechas:
- La función `format_date_for_influxdb` se encarga de asignar un timezone a una fecha para finalmente devolver la fecha con timezone 'UTC'. Si le llega una fecha sin timezone entiende que se refiere al timezone local. El timezone local es el correspondiente a la máquina que corre Home Assistant, y en el custom component esta información se ha almacenado en `self.class_local_timezone` en la inicialización del coordinador. En un custom component cualquiera, el string del timezone de la máquina puede obtenerse de `hass.config.time_zone`. Posteriormente este string puede ser convertido mediante `class_local_timezone=pytz.timezone(hass.config.time_zone)`.
- El custom component generará consultas utilizando `energy_query_string`. El formato de fechas que se pasará a `energy_query_string` en principio deberán ser fechas con timezone, sin embargo, dado que esta función llama a la función `format_date_for_influxdb`, aunque se pasen las fechas sin timezone, la consulta se construirá con rangos de fechas con timezone UTC, que es lo que requiere InfluxDB.
- Una vez construida la consulta, utilizando `client=InfluxDBClient(...)` se establecerá la conexión con InfluxDB y se realizará la consulta mediante `result = client.query(str_query)`. Esto devolverá un json donde las fechas llegarán como strings con timezone UTC.
- El json `result` se extrae al diccionario `points` mediante `points = list(result.get_points())`. Este diccionario tendrá como clave los string con las fechas localizadas en UTC.
- Será necesario generar un dataframe de Pandas ya que se va a realizar operaciones para las que resulta conveniente operar con dataframes. El dataframe se genera mediante `df1 = delta_energy_dataframe(points)`. 
- La función `delta_energy_dataframe(points)` se encarga de convertir un diccionario con valores absolutos de energía acumulada, en valores diferenciales en cada intervalo (delta). Esto conlleva la perdida de la primera fecha. Los valores delta son devueltos convertidos en un dataframe donde el índice es la fecha, todavía con información de timezone UTC que llegó en `points`. Las instrucciones son una primera aproximación a lo que podría ser `delta_energy_dataframe(points)`:
```python
df = pd.DataFrame(points)
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None) # Eliminate the timezone for Prophet
df.set_index('time', inplace=True)
first_index = df['energy_kWh'].first_valid_index() # first index where the value is not NaN
first_value = df.loc[first_index, 'energy_kWh'] # first value that is not NaN
mask = df.index < first_index # records before the first non-NaN value
df.loc[mask, 'energy_kWh'] = first_value  # Fill NaN values with first_value
df['delta_energy'] = df['energy_kWh'].diff() # Calculate the difference between consecutive values
# Remove the first row of the DataFrame (the hourly difference of the first record does not make sense)
df = df.dropna()
# If there has been any counter reset, the difference will be negative
mask = df['delta_energy'] < 0
df.loc[mask, 'delta_energy'] = 0 # In those cases, set the difference to 0
return df
```
- La llamada a `delta_energy_dataframe(points)` se realiza dos veces si `str_query2 is not None`. En este caso tendremos df1 y df2 y se sumarán mediante la sucesión de instrucciones:
```python
df = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
df['delta_energy'] = df['delta_energy_x'].fillna(0) + df['delta_energy_y'].fillna(0)
df['delta_energy'] = df['delta_energy'].fillna(0) 
df.reset_index(inplace=True)                 
df = df[['time', 'delta_energy']]
```
- Con el df resultante se entrenará el modelo Prophet, será la siguiente sucesión de instrucciones:
```python
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds']).dt.tz_localize(None) # Se ha hecho en delta_energy_dataframe
model = Prophet()
model.fit(df)
```
- Una vez entrenado el modelo, se genera un nuevo dataframe `future` que especifica `futurePeriods` adicionales para realizar las previsiones. 
```python
future = model.make_future_dataframe(periods=futurePeriods, freq=futureFreq)
```
- A partir de `future` se genera finalmente el dataframe `forecast` que contiene todo el rango de datos, es decir, tanto los datos del entrenamiento como los futurePeriods siguientes al periodo de entrenamiento y que por tanto son la predicción buscada. En primer lugar se recorta el datframe mediante `forecast=forecast.tail(futurePeriods)`. Este dataframe requiere incorporar la información del timezone para que cuando regrese al custom component sea posible darle utilidad. Este trabajo que se realiza dentro del contenedor Docker, no tiene acceso a la zona horaria local por lo que la mejor opción es devolver los datos en UTC. El servidor FastAPI enviará los resultados al usuario en formato json que obtendrá del diccionario generado en `response`
```python
forecast = model.predict(future)
forecast=forecast.tail(futurePeriods)
forecast['ds'] = pd.to_datetime(forecast['ds']).dt.tz_localize('UTC')
forecast = forecast.set_index('ds')
response = forecast.to_dict()['yhat']
```

En los apartados siguientes se irá probando está estrategia y se justificará porque finalmente no se adopta exactamente como estaba planteada inicialmente.

## Comprobar el funcionamiento de `format_date_for_influxdb`

In [19]:
# Prueba de format_date_for_influxdb
date = '2024-10-01' #Fecha sin zona horaria ni horas o minutos o segundos, por lo que se asume T00:00:00
print(date)
new_date = format_date_for_influxdb(date)
print(new_date)

2024-10-01
2024-09-30T22:00:00Z


In [20]:
# Prueba de format_date_for_influxdb
date = '2024-10-01T00:00:00' #Fecha sin zona horaria
print(date)
new_date = format_date_for_influxdb(date)
print(new_date)


2024-10-01T00:00:00
2024-09-30T22:00:00Z


In [21]:
# Prueba de format_date_for_influxdb
date = datetime.now() #Fecha sin zona horaria
print(date)
new_date = format_date_for_influxdb(date)
print(new_date)

2024-11-17 18:49:41.654326
2024-11-17T17:49:41Z


In [22]:
# Prueba de format_date_for_influxdb
print(class_local_timezone.tzname)
date = datetime.now(class_local_timezone) #Fecha con zona horaria
print(date)
new_date = format_date_for_influxdb(date)
print(new_date)

<bound method DstTzInfo.tzname of <DstTzInfo 'Europe/Madrid' LMT-1 day, 23:45:00 STD>>
2024-11-17 18:49:41.666415+01:00
2024-11-17T17:49:41Z


In [23]:
class_local_timezone=pytz.timezone('America/New_York')
date = datetime.now(class_local_timezone) #Fecha con zona horaria
print(date)

2024-11-17 12:49:41.679916-05:00


Como se observa la fecha ahora sí tiene información de timezone (-05:00) Esto se puede comprobar mediante `if date.tzinfo is None:`

In [24]:
if date.tzinfo is None:
    print("No tiene zona horaria")
else:
    print("Tiene zona horaria")

Tiene zona horaria


A la función `format_date_for_influxdb` también se le pueden pasar fechas localizadas y las devolverá en UTC y eliminado los deciamles de los segundos que no serán relevantes para realizar la consulta de energía

In [25]:
new_date = format_date_for_influxdb(date)
print(new_date)

2024-11-17T17:49:41Z


Cuando una fecha ya tiene información de zona horaria, no se puede localizar, solo se puede convertir. Por tanto lo siguiente genera un error.

In [26]:
# Descomentar para provocar un error para ilustrar que las fechas ya localizadas no se pueden volver a localizar
date = pd.to_datetime(date)
# date = date.tz_localize('America/New_York')

Podemos localizar date en otra zona horaria

In [27]:
print(f"'America/New_York': {date}")
# Para poder usar tz_convert date debe ser un objeto de tipo Timestamp
date = pd.to_datetime(date)
date = date.tz_convert('UTC')
print(f"'UTC': {date}")
class_local_timezone=pytz.timezone('Europe/Madrid')
date = date.tz_convert(class_local_timezone)
print(f"'Europe/Madrid': {date}")

'America/New_York': 2024-11-17 12:49:41.679916-05:00
'UTC': 2024-11-17 17:49:41.679916+00:00
'Europe/Madrid': 2024-11-17 18:49:41.679916+01:00


In [28]:
new_date = format_date_for_influxdb(date)
print(new_date)

2024-11-17T17:49:41Z


## Prueba de la función `energy_query_string`
def energy_query_string(entity_id, start: str | None = None, end: str | None = None) -> str:

In [29]:
str_query1 = energy_query_string('victron_vebus_acin1toacout_228') # Sin especificar fechas
str_query1

'SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE "entity_id"=\'victron_vebus_acin1toacout_228\' GROUP BY time(1h) fill(previous)'

In [30]:

str_query1 = energy_query_string('victron_vebus_acin1toacout_228',end=new_date) # Con fecha final
str_query1

'SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time <= \'2024-11-17T17:49:41Z\') AND "entity_id"=\'victron_vebus_acin1toacout_228\' GROUP BY time(1h) fill(previous)'

In [31]:
start=pd.to_datetime(new_date) - timedelta(days=1)
str_query1 = energy_query_string('victron_vebus_acin1toacout_228',start=start, end=new_date) # Con fecha inicial y final
str_query1

'SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time >= \'2024-11-16T17:49:41Z\') AND (time <= \'2024-11-17T17:49:41Z\') AND "entity_id"=\'victron_vebus_acin1toacout_228\' GROUP BY time(1h) fill(previous)'

## Definir el proceso analizando las instrucciones paso a paso
Tras las pruebas realizadas en el código de los siguientes apartados se llega a la siguiente conclusión que será necesario tener en cuenta no solo para la creación del addon sino también para la utilización del mismo desde el custom component.
**Conclusión**: La mejor forma de componer la consulta, para consultas de energía desde el custom component es especificar siempre la hora de fin igual a la hora actual.
Esto asegura que la fecha/hora correspondiente al momento actual estará presente en el conjunto de puntos devuelto por Influx, muy probablemente con un valor NaN ya que todavía no habrá podido consolidar datos para la hora actual. De esta forma podemos asegurar que si se elimina el último registro, la primera predicción que hará Prophet sea la correspondiente a la hora actual.

In [32]:
print(f"Host: {host}, Port: {port}, User: {user}, Database: {dbname}")
# Bloque comentado tras las pruebas
# ***********************************************************************************************************************
start='2023-09-27T15:00:00Z'
#end='2023-10-27T15:00:00Z'
end=datetime.now()
# start=pd.to_datetime(end) - timedelta(days=1)
str_query1 = energy_query_string('victron_vebus_acin1toacout_228',start=start, end=end) # Con fecha inicial y final
#str_query1 = energy_query_string('victron_vebus_acin1toacout_228', end=end) # Con fecha final, Devuelve todo lo anterior si tiene datos, sino vació
#str_query1 = energy_query_string('victron_vebus_acin1toacout_228') # Sin especificar fechas. Devuelve todo si tiene datos, sino vació
# **********************************************************************************************************************
# Conclusión final
end=datetime.now()
start=None # start se podría cambiar a un valor para limitar con cuantos datos se entrenará el modelo. 
# Sin embargo si se usa start se debería asegurar que hay datos después de start ya que sino rellenará con NaN. los primeros registros que finalmente convertiremos a cero
str_query1 = energy_query_string('victron_vebus_acin1toacout_228',start=start, end=end) # Con fecha inicial=None y final
print(f"Query: {str_query1}")
client = InfluxDBClient(host=host, port=port, username=user, password=password, database=dbname)
result = client.query(str_query1) # Si hay datos en el rango, será un conjunto de valores continuo, sin existir huecos entre fechas
points = list(result.get_points())
# Mostar los primeros 5 registros
points[:5]

Host: 192.168.0.100, Port: 8086, User: homeassistant, Database: homeassistant
Query: SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time <= '2024-11-17T17:49:41Z') AND "entity_id"='victron_vebus_acin1toacout_228' GROUP BY time(1h) fill(previous)


[{'time': '2024-09-27T15:00:00Z', 'energy_kWh': 94.54},
 {'time': '2024-09-27T16:00:00Z', 'energy_kWh': 94.64},
 {'time': '2024-09-27T17:00:00Z', 'energy_kWh': 94.68},
 {'time': '2024-09-27T18:00:00Z', 'energy_kWh': 94.83},
 {'time': '2024-09-27T19:00:00Z', 'energy_kWh': 94.9}]

In [33]:
# Mostrar el largo de la lista de puntos
print(f"Longitud de la lista de puntos: {len(points)}")

Longitud de la lista de puntos: 1227


Se observa como a pesar de que la fecha/hora incluye minutos y segundos no se tiene en cuenta al estar la clausula `GROUP BY time(1h)` y ha devuelto también el resultado de las T13:00:00Z.

Por otra parte, se debe tener en cuenta que si se han especificado ambas fechas start y end en la consulta y en este rango de fechas seleccionadas hay algún dato, la respuesta será un conjunto de datos continuo entre ambas fechas con el incremento de la clausula Group By, en este caso 1h.
Si no hay ningún dato devuelve un dataset vacio

Los datos han llegado como valores acumulados desde el último reset del contador, vamos ahora a realizar el procesamiento que será necesario para convertirlos en valores por hora utilizando dataframes.

In [34]:
df = pd.DataFrame(points)
df.tail()

,time,energy_kWh
1222,2024-11-17T13:00:00Z,11.91
1223,2024-11-17T14:00:00Z,12.29
1224,2024-11-17T15:00:00Z,12.60
1225,2024-11-17T16:00:00Z,12.62
1226,2024-11-17T17:00:00Z,12.63


In [35]:
df.describe()

,energy_kWh
count,1227.000000
mean,50.849340
std,36.907061
min,0.000000
25%,12.610000
50%,49.590000
75%,84.160000
max,117.640000


In [36]:
# Mostrar el largo del dataframe
print(f"Longitud del dataframe: {len(df)}")

Longitud del dataframe: 1227


Observar que `df.describe()` no tiene en cuenta los NaN pero las filas existen.

In [37]:
df['time'] = pd.to_datetime(df['time'])
df.tail()

,time,energy_kWh
1222,2024-11-17 13:00:00+00:00,11.91
1223,2024-11-17 14:00:00+00:00,12.29
1224,2024-11-17 15:00:00+00:00,12.60
1225,2024-11-17 16:00:00+00:00,12.62
1226,2024-11-17 17:00:00+00:00,12.63


In [38]:
df['time'] = df['time'].dt.tz_localize(None)
df.tail()

,time,energy_kWh
1222,2024-11-17 13:00:00,11.91
1223,2024-11-17 14:00:00,12.29
1224,2024-11-17 15:00:00,12.60
1225,2024-11-17 16:00:00,12.62
1226,2024-11-17 17:00:00,12.63


In [39]:
# Todo lo anterior se podría hacer en un solo paso
df = pd.DataFrame(points)
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
df.tail()

,time,energy_kWh
1222,2024-11-17 13:00:00,11.91
1223,2024-11-17 14:00:00,12.29
1224,2024-11-17 15:00:00,12.60
1225,2024-11-17 16:00:00,12.62
1226,2024-11-17 17:00:00,12.63


Para poder hacer operaciones basadas en el tiempo necesitaremos que time sea el indice

In [40]:
df.set_index('time', inplace=True)
df.tail()

,energy_kWh
time,
2024-11-17 13:00:00,11.91
2024-11-17 14:00:00,12.29
2024-11-17 15:00:00,12.60
2024-11-17 16:00:00,12.62
2024-11-17 17:00:00,12.63


Puede suceder que en el rango de fechas seleccionadas existan valores NaN
vamos a mostrar si hay alguno

In [41]:
# Mostrar df donde haya NaN
df[df.iloc[:,0].isnull()] # Notación de pandas para seleccionar columnas por índice

,energy_kWh
time,


In [42]:
# Mostrar df donde haya NaN
df[df.iloc[:,0].isnull()]


,energy_kWh
time,


Vamos a forzar algunos valores NaN

In [43]:
# Forzar algunos valores Nan para probar
# En el primer registro
df.iloc[0,0] = np.nan
# En el quinto registro
df.iloc[4,0] = np.nan
# En el último registro
df.iloc[-1,0] = np.nan
df

,energy_kWh
time,
2024-09-27 15:00:00,NaN
2024-09-27 16:00:00,94.64
2024-09-27 17:00:00,94.68
2024-09-27 18:00:00,94.83
2024-09-27 19:00:00,NaN
...,...
2024-11-17 13:00:00,11.91
2024-11-17 14:00:00,12.29
2024-11-17 15:00:00,12.60


Para tratar los posibles valores NaN iniciales se podría usar lo siguiente, sin embargo se llega a lo conclusión de que esto podría falsear los datos y finalmente no se implementará. 

In [44]:
df.tail()

,energy_kWh
time,
2024-11-17 13:00:00,11.91
2024-11-17 14:00:00,12.29
2024-11-17 15:00:00,12.60
2024-11-17 16:00:00,12.62
2024-11-17 17:00:00,NaN


In [45]:
first_index = df.iloc[:,0].first_valid_index() # first index where the value is not NaN
first_index

Timestamp('2024-09-27 16:00:00')

In [46]:
# Valor de la columna 0 correspondiente a first_index
first_value=df.loc[first_index, df.columns[0]]
first_value

np.float64(94.64)

Se ha observado un reset del contador de energía

In [47]:
mask = df.index < first_index # records before the first non-NaN value
mask

array([ True, False, False, ..., False, False, False])

In [48]:
df.loc[mask, df.columns[0]] = first_value  # Fill NaN values with first_value
df

,energy_kWh
time,
2024-09-27 15:00:00,94.64
2024-09-27 16:00:00,94.64
2024-09-27 17:00:00,94.68
2024-09-27 18:00:00,94.83
2024-09-27 19:00:00,NaN
...,...
2024-11-17 13:00:00,11.91
2024-11-17 14:00:00,12.29
2024-11-17 15:00:00,12.60


Ahora vamos a calcular las diferencias para tener valor de consumo por hora

In [49]:
# Hacer diff() sin conocer el nombre de la columna por su indice
df['delta_energy'] = df.iloc[:,0].diff() # Calculate the difference between consecutive values
df

,energy_kWh,delta_energy
time,,
2024-09-27 15:00:00,94.64,NaN
2024-09-27 16:00:00,94.64,0.00
2024-09-27 17:00:00,94.68,0.04
2024-09-27 18:00:00,94.83,0.15
2024-09-27 19:00:00,NaN,NaN
...,...,...
2024-11-17 13:00:00,11.91,0.53
2024-11-17 14:00:00,12.29,0.38
2024-11-17 15:00:00,12.60,0.31


Como se observa el primer valor es NaN ya que ha hecho la diferencia con un valor anterior no existente. Del mismo modo los valores que inicialmente eran NaN han provocado varios nuevos NaN en la columna delta_energy. 
Eliminar todos los NaN

In [50]:
df = df.dropna()
df

,energy_kWh,delta_energy
time,,
2024-09-27 16:00:00,94.64,0.00
2024-09-27 17:00:00,94.68,0.04
2024-09-27 18:00:00,94.83,0.15
2024-09-27 21:00:00,95.30,0.29
2024-09-27 22:00:00,95.52,0.22
...,...,...
2024-11-17 12:00:00,11.38,0.69
2024-11-17 13:00:00,11.91,0.53
2024-11-17 14:00:00,12.29,0.38


Hemos perdido algunas franjas horarias

# Conclusiones y reinicio de los datos para pasar a la siguiente etapa
Vamos a intentar no hacer un tratamiento excesivo de los valores NaN para ver como responde Prophet en el entrenamiento cuando recibe valores NaN.
Por otra parte la conclusión más relevante, es que al generar la consulta es conveniente especificar la fecha `end` ya que de esta forma se asegura que el momento presente queda en el dataset y aunque después se eliminará por no tener datos consolidados. Permite coordinar cual será el primer registro del forecast entre el Custom Component y el Addon, permitiendo que el primer registro devuelto en el forecast sea precisamente el pronostico de la hora actual.
Con estas premisas se prosigue la prueba para obtener una secuencia de instrucciones óptima.

In [51]:
print(f"Host: {host}, Port: {port}, User: {user}, Database: {dbname}")
end=datetime.now()
str_query1 = energy_query_string('victron_vebus_acin1toacout_228',end=end) # Solo fecha final igual a la actual
print(f"Query: {str_query1}")
client = InfluxDBClient(host=host, port=port, username=user, password=password, database=dbname)
result = client.query(str_query1) # Si hay datos en el rango, será un conjunto de valores continuo, sin existir huecos entre fechas (podría haber valores NaN)
points = list(result.get_points())
df = pd.DataFrame(points)
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
df.set_index('time', inplace=True)
df['delta_energy'] = df.iloc[:,0].diff() # Calculate the difference between consecutive values
# ¿Como responde Prophet si recibe un registro con NaN? probamos a comentar la siguiente línea
# df = df.drop(df.index[0]) # eliminar la fila del primer registro 
df = df.drop(df.index[-1]) # eliminar la última fila. Valor no consolidado en la hora actual
# mostrar valores <0 para ver si ha habido reset del contador
df[df['delta_energy']<0]


Host: 192.168.0.100, Port: 8086, User: homeassistant, Database: homeassistant
Query: SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time <= '2024-11-17T17:49:42Z') AND "entity_id"='victron_vebus_acin1toacout_228' GROUP BY time(1h) fill(previous)


,energy_kWh,delta_energy
time,,
2024-10-05 09:00:00,0.18,-117.46
2024-11-09 07:00:00,0.00,-100.18


In [52]:


# If there has been any counter reset, the difference will be negative
mask = df['delta_energy'] < 0
# ¿Que asignar 0 o NaN? --> Asignar NaN para ver como se comporta Prophet
df.loc[mask, 'delta_energy'] = np.nan # In those cases, set the difference to NaN
# df.loc[mask, 'delta_energy'] = 0 # In those cases, set the difference to 0
df.head()

,energy_kWh,delta_energy
time,,
2024-09-27 15:00:00,94.54,NaN
2024-09-27 16:00:00,94.64,0.10
2024-09-27 17:00:00,94.68,0.04
2024-09-27 18:00:00,94.83,0.15
2024-09-27 19:00:00,94.90,0.07


In [53]:
df.tail()

,energy_kWh,delta_energy
time,,
2024-11-17 12:00:00,11.38,0.69
2024-11-17 13:00:00,11.91,0.53
2024-11-17 14:00:00,12.29,0.38
2024-11-17 15:00:00,12.60,0.31
2024-11-17 16:00:00,12.62,0.02


Suponer que hay un segundo valor que se debe sumar --> df2

In [54]:
df1=df.copy()


In [55]:

str_query2 = energy_query_string('victron_vebus_invertertoacout_228',end=end) # Solo fecha final igual a la actual
print(f"Query: {str_query2}")
client = InfluxDBClient(host=host, port=port, username=user, password=password, database=dbname)
result = client.query(str_query2) # Si hay datos en el rango, será un conjunto de valores continuo, sin existir huecos entre fechas
points = list(result.get_points())
df = pd.DataFrame(points)
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)
df.set_index('time', inplace=True)
df['delta_energy'] = df.iloc[:,0].diff() # Calculate the difference between consecutive values
# ¿Como responde Prophet si recibe un registro con NaN? probamos a comentar la siguiente línea
# df = df.drop(df.index[0]) # eliminar la fila del primer registro 
df = df.drop(df.index[-1]) # eliminar la última fila. Valor no consolidado en la hora actual
# mostrar valores <0 para ver si ha habido reset del contador
df[df['delta_energy']<0]

Query: SELECT last("value") AS "energy_kWh" FROM "kWh" WHERE (time <= '2024-11-17T17:49:42Z') AND "entity_id"='victron_vebus_invertertoacout_228' GROUP BY time(1h) fill(previous)


,energy_kWh,delta_energy
time,,
2024-10-05 09:00:00,0.15,-22.70
2024-11-09 07:00:00,0.00,-38.28


In [56]:

# If there has been any counter reset, the difference will be negative
mask = df['delta_energy'] < 0
# ¿Que asignar 0 o NaN? --> Asignar NaN para ver como se comporta Prophet
df.loc[mask, 'delta_energy'] = np.nan # In those cases, set the difference to NaN
# df.loc[mask, 'delta_energy'] = 0 # In those cases, set the difference to 0
df.head()

,energy_kWh,delta_energy
time,,
2024-09-27 15:00:00,16.98,NaN
2024-09-27 16:00:00,16.98,0.00
2024-09-27 17:00:00,17.02,0.04
2024-09-27 18:00:00,17.02,0.00
2024-09-27 19:00:00,17.02,0.00


In [57]:
df2=df.copy()
print(len(df1))
# Eliminar un par de registros de df2 para incordiar
df2 = df2.drop(df2.index[-1])
df2 = df2.drop(df2.index[-1])
print(len(df2))

1226
1224


In [58]:
# Juntar los dos dataframes indicando que los indices de ambos son válidos
df = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
df.tail()

,energy_kWh_x,delta_energy_x,energy_kWh_y,delta_energy_y
time,,,,
2024-11-17 12:00:00,11.38,0.69,17.15,0.55
2024-11-17 13:00:00,11.91,0.53,17.55,0.40
2024-11-17 14:00:00,12.29,0.38,17.73,0.18
2024-11-17 15:00:00,12.60,0.31,NaN,NaN
2024-11-17 16:00:00,12.62,0.02,NaN,NaN


In [59]:
# Sumar delta_energy_x y delta_energy_y reemplazando los NaN por ceros
# De esta forma si alguno de los sensores ha fallado al menos se tendrá el valor del otro
df['delta_energy'] = df['delta_energy_x'].fillna(0) + df['delta_energy_y'].fillna(0)
df.tail()

,energy_kWh_x,delta_energy_x,energy_kWh_y,delta_energy_y,delta_energy
time,,,,,
2024-11-17 12:00:00,11.38,0.69,17.15,0.55,1.24
2024-11-17 13:00:00,11.91,0.53,17.55,0.40,0.93
2024-11-17 14:00:00,12.29,0.38,17.73,0.18,0.56
2024-11-17 15:00:00,12.60,0.31,NaN,NaN,0.31
2024-11-17 16:00:00,12.62,0.02,NaN,NaN,0.02


In [60]:
df['delta_energy'] = df['delta_energy_x'] + df['delta_energy_y']
df.tail()

,energy_kWh_x,delta_energy_x,energy_kWh_y,delta_energy_y,delta_energy
time,,,,,
2024-11-17 12:00:00,11.38,0.69,17.15,0.55,1.24
2024-11-17 13:00:00,11.91,0.53,17.55,0.40,0.93
2024-11-17 14:00:00,12.29,0.38,17.73,0.18,0.56
2024-11-17 15:00:00,12.60,0.31,NaN,NaN,NaN
2024-11-17 16:00:00,12.62,0.02,NaN,NaN,NaN


In [61]:
# Dejar df solo con las columnas time y delta_energy
# Hacer que time deje de ser el índice
df.reset_index(inplace=True)
df = df[['time', 'delta_energy']]
df.tail()

,time,delta_energy
1221,2024-11-17 12:00:00,1.24
1222,2024-11-17 13:00:00,0.93
1223,2024-11-17 14:00:00,0.56
1224,2024-11-17 15:00:00,NaN
1225,2024-11-17 16:00:00,NaN


In [62]:
# Provocar valores NaN en delta_energy en algunos índices
df.loc[100,'delta_energy'] = np.nan
df.loc[200,'delta_energy'] = np.nan
df.loc[300,'delta_energy'] = np.nan
df.loc[400,'delta_energy'] = np.nan
df.loc[600,'delta_energy'] = np.nan
# Mostrar si hay valores NaN en delta_energy
df[df['delta_energy'].isnull()]


,time,delta_energy
0,2024-09-27 15:00:00,NaN
100,2024-10-01 19:00:00,NaN
186,2024-10-05 09:00:00,NaN
200,2024-10-05 23:00:00,NaN
300,2024-10-10 03:00:00,NaN
400,2024-10-14 07:00:00,NaN
600,2024-10-22 15:00:00,NaN
1024,2024-11-09 07:00:00,NaN
1224,2024-11-17 15:00:00,NaN
1225,2024-11-17 16:00:00,NaN


# Entrenar el modelo Prophet y hacer predicciones

In [63]:
df.columns = ['ds', 'y']
model = Prophet()
model.fit(df)

18:49:43 - cmdstanpy - INFO - Chain [1] start processing
18:49:43 - cmdstanpy - INFO - Chain [1] done processing


In [64]:
futurePeriods=12
futureFreq='h'
future = model.make_future_dataframe(periods=futurePeriods, freq=futureFreq)
future.head()

,ds
0,2024-09-27 15:00:00
1,2024-09-27 16:00:00
2,2024-09-27 17:00:00
3,2024-09-27 18:00:00
4,2024-09-27 19:00:00


In [65]:
future.tail(futurePeriods)

,ds
1226,2024-11-17 17:00:00
1227,2024-11-17 18:00:00
1228,2024-11-17 19:00:00
1229,2024-11-17 20:00:00
1230,2024-11-17 21:00:00
1231,2024-11-17 22:00:00
1232,2024-11-17 23:00:00
1233,2024-11-18 00:00:00
1234,2024-11-18 01:00:00
1235,2024-11-18 02:00:00


Se comprueba que si hacemos tail con la misma cantidad de periodos que se ha indicado en futurePeriods la primera predicción será para la hora actual

In [66]:
forecast = model.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-09-27 15:00:00,0.157165,-0.041838,0.335140,0.157165,0.157165,-0.020493,-0.020493,-0.020493,-0.006046,-0.006046,-0.006046,-0.014447,-0.014447,-0.014447,0.0,0.0,0.0,0.136672
1,2024-09-27 16:00:00,0.157179,-0.088546,0.325878,0.157179,0.157179,-0.039015,-0.039015,-0.039015,-0.023654,-0.023654,-0.023654,-0.015361,-0.015361,-0.015361,0.0,0.0,0.0,0.118164
2,2024-09-27 17:00:00,0.157192,-0.068160,0.339339,0.157192,0.157192,-0.024450,-0.024450,-0.024450,-0.008193,-0.008193,-0.008193,-0.016256,-0.016256,-0.016256,0.0,0.0,0.0,0.132743
3,2024-09-27 18:00:00,0.157206,-0.064821,0.352894,0.157206,0.157206,-0.002592,-0.002592,-0.002592,0.014537,0.014537,0.014537,-0.017129,-0.017129,-0.017129,0.0,0.0,0.0,0.154614
4,2024-09-27 19:00:00,0.157220,-0.035590,0.370651,0.157220,0.157220,0.002961,0.002961,0.002961,0.020939,0.020939,0.020939,-0.017978,-0.017978,-0.017978,0.0,0.0,0.0,0.160181


In [67]:
# Comprobar si hay valores NaN en forecast y en que columnas
forecast.isnull().sum()


ds                            0
trend                         0
yhat_lower                    0
yhat_upper                    0
trend_lower                   0
trend_upper                   0
additive_terms                0
additive_terms_lower          0
additive_terms_upper          0
daily                         0
daily_lower                   0
daily_upper                   0
weekly                        0
weekly_lower                  0
weekly_upper                  0
multiplicative_terms          0
multiplicative_terms_lower    0
multiplicative_terms_upper    0
yhat                          0
dtype: int64

Se comprueba que a pesar de los valores NaN, Prophet a hecho un pronostico sin NaN. Por tanto en la medida de que no estemos seguros de como tratar los datos es preferible dejar valores NaN que asignar datos no confiables, ya que Prophet seguirá haciendo bien su pronostico.

In [68]:
forecast=forecast.tail(futurePeriods)
forecast['ds'] = pd.to_datetime(forecast['ds']).dt.tz_localize('UTC')
forecast = forecast.set_index('ds')
forecast

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
ds,,,,,,,,,,,,,,,,,,
2024-11-17 17:00:00+00:00,0.161267,-0.019934,0.373903,0.161267,0.161267,0.010137,0.010137,0.010137,-0.008193,-0.008193,-0.008193,0.018330,0.018330,0.018330,0.0,0.0,0.0,0.171403
2024-11-17 18:00:00+00:00,0.161262,-0.004387,0.394732,0.161262,0.161262,0.034702,0.034702,0.034702,0.014537,0.014537,0.014537,0.020165,0.020165,0.020165,0.0,0.0,0.0,0.195964
2024-11-17 19:00:00+00:00,0.161257,0.004566,0.402785,0.161257,0.161257,0.042847,0.042847,0.042847,0.020939,0.020939,0.020939,0.021908,0.021908,0.021908,0.0,0.0,0.0,0.204104
2024-11-17 20:00:00+00:00,0.161252,-0.016173,0.393063,0.161252,0.161252,0.032314,0.032314,0.032314,0.008766,0.008766,0.008766,0.023547,0.023547,0.023547,0.0,0.0,0.0,0.193566
2024-11-17 21:00:00+00:00,0.161247,-0.013953,0.369390,0.161247,0.161247,0.017315,0.017315,0.017315,-0.007757,-0.007757,-0.007757,0.025072,0.025072,0.025072,0.0,0.0,0.0,0.178562
2024-11-17 22:00:00+00:00,0.161243,-0.034389,0.373176,0.161243,0.161243,0.008228,0.008228,0.008228,-0.018243,-0.018243,-0.018243,0.026472,0.026472,0.026472,0.0,0.0,0.0,0.169471
2024-11-17 23:00:00+00:00,0.161238,-0.047571,0.372940,0.161238,0.161238,-0.000641,-0.000641,-0.000641,-0.028377,-0.028377,-0.028377,0.027736,0.027736,0.027736,0.0,0.0,0.0,0.160597
2024-11-18 00:00:00+00:00,0.161233,-0.075926,0.338269,0.161233,0.161233,-0.021991,-0.021991,-0.021991,-0.050847,-0.050847,-0.050847,0.028856,0.028856,0.028856,0.0,0.0,0.0,0.139242
2024-11-18 01:00:00+00:00,0.161228,-0.104859,0.307205,0.161228,0.161228,-0.055859,-0.055859,-0.055859,-0.085684,-0.085684,-0.085684,0.029825,0.029825,0.029825,0.0,0.0,0.0,0.105369


In [69]:
response = forecast.to_dict()['yhat']
response


{Timestamp('2024-11-17 17:00:00+0000', tz='UTC'): 0.17140345014419348,
 Timestamp('2024-11-17 18:00:00+0000', tz='UTC'): 0.19596385518484175,
 Timestamp('2024-11-17 19:00:00+0000', tz='UTC'): 0.20410424649384762,
 Timestamp('2024-11-17 20:00:00+0000', tz='UTC'): 0.19356591276131851,
 Timestamp('2024-11-17 21:00:00+0000', tz='UTC'): 0.17856223701165808,
 Timestamp('2024-11-17 22:00:00+0000', tz='UTC'): 0.1694710169993906,
 Timestamp('2024-11-17 23:00:00+0000', tz='UTC'): 0.16059686256482147,
 Timestamp('2024-11-18 00:00:00+0000', tz='UTC'): 0.13924229122202675,
 Timestamp('2024-11-18 01:00:00+0000', tz='UTC'): 0.10536921275747424,
 Timestamp('2024-11-18 02:00:00+0000', tz='UTC'): 0.07885713362647047,
 Timestamp('2024-11-18 03:00:00+0000', tz='UTC'): 0.08389865950960555,
 Timestamp('2024-11-18 04:00:00+0000', tz='UTC'): 0.1247033982638879}

In [70]:
# Cuando el custom component reciba estos datos deberá convertirlos a la hora local definida en class_local_timezone 
response = {str(k): v for k, v in response.items()}
# Convertir las claves a la hora local
response = {pd.to_datetime(k).tz_convert(class_local_timezone).strftime('%Y-%m-%dT%H:%M:%S'): v for k, v in response.items()}
response

{'2024-11-17T18:00:00': 0.17140345014419348,
 '2024-11-17T19:00:00': 0.19596385518484175,
 '2024-11-17T20:00:00': 0.20410424649384762,
 '2024-11-17T21:00:00': 0.19356591276131851,
 '2024-11-17T22:00:00': 0.17856223701165808,
 '2024-11-17T23:00:00': 0.1694710169993906,
 '2024-11-18T00:00:00': 0.16059686256482147,
 '2024-11-18T01:00:00': 0.13924229122202675,
 '2024-11-18T02:00:00': 0.10536921275747424,
 '2024-11-18T03:00:00': 0.07885713362647047,
 '2024-11-18T04:00:00': 0.08389865950960555,
 '2024-11-18T05:00:00': 0.1247033982638879}

## Replicar las funciones del contenedor Docker
Todas las pruebas anteriores van a servir para finalmente dar forma a las funciones que formarán parte del archivo `main.py` que correrá el contenedor del addon **Prophet InfluxDB Addon**.
La función `post_energy_queries` replica el funcionamiento que realizará el endpoint `@app.post("/energy_queries")` con la función `async def query(request: EnergyQueryRequest)` en `main.py`.

In [71]:
def delta_energy_dataframe(points) -> pd.DataFrame:
    """ The query must have GROUP BY time('time(1h)'). Normally 'h' but can be changed to other time intervals.
    points come in UTC timezone.
    """
    try:
        df = pd.DataFrame(points)
        
        if 'time' not in df.columns:
            raise ValueError("Column 'time' not found in the input data")
        
        df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None) # Eliminate the timezone for Prophet
                
        # Convert time to the index of the DataFrame
        df.set_index('time', inplace=True)
        
        if df.empty:
            raise ValueError("DataFrame is empty after setting 'time' as index")
                
        # Create a new column that calculates the hourly energy difference
        df['delta_energy'] = df.iloc[:,0].diff() # Calculate the difference between consecutive values
        df = df.drop(df.index[0]) # Remove the first row of the DataFrame (the hourly difference of the first record does not make sense)
        df = df.drop(df.index[-1]) # Remove the last row of the DataFrame (there isn´t enough data yet in the last hour)
                
        # If there has been any counter reset, the difference will be negative
        mask = df['delta_energy'] < 0
        # The next line is replaced because Prophet can manage perfectly the NaN values
        # df.loc[mask, 'delta_energy'] = 0 # In those cases, set the difference to 0
        df.loc[mask, 'delta_energy'] = np.nan # In those cases, set the difference to NaN
        
        return df
    
    except Exception as e:
        logging.error(f"Error processing dataframes: {e}")

def post_energy_queries(str_query1: str, str_query2: str = None, \
                        futurePeriods: int=30, futureFreq: str ='H') -> pd.DataFrame:
    """ Post a query to the InfluxDB database and return the result as a DataFrame """
    # Simula la función 
    # @app.post("/energy_queries")
    # async def query(request: EnergyQueryRequest):


    # Check that the query has the groupby('time(1h)') clause
    if 'GROUP BY TIME(' not in str_query1.upper():
        print("The query must have GROUP BY time('time(1h)')")
        return None
        #raise HTTPException(status_code=400, detail="The query must have GROUP BY time('time(1h)')")
    if str_query2 is not None and 'GROUP BY TIME(' not in str_query2.upper():
        print("The query must have GROUP BY time('time(1h)')")
        return None
        #raise HTTPException(status_code=400, detail="The query must have GROUP BY time('time(1h)')")
    try:
        # Connect to InfluxDB
        client = InfluxDBClient(host=host, port=port, username=user, password=password, database=dbname)
        logging.debug("Connected to InfluxDB step 1")

    except Exception as e:
        print(f"Error connecting to InfluxDB step 1: {str(e)}")
        return None
        #raise HTTPException(status_code=400, detail=f"Error connecting to InfluxDB step 1: {str(e)}")
    
    try:
        # Execute the query
        result = client.query(str_query1)
        logging.debug("Query_1 executed successfully")
        # Convert the result to a DataFrame
        points = list(result.get_points()) # Get dates in UTC
    except Exception as e:
        print(f"Error executing query1: {str(e)}")
        return None
        #raise HTTPException(status_code=400, detail=f"Error executing query1: {str(e)}")

    
    if not points:
        print("No data returned from query1")
        return None
        #raise HTTPException(status_code=400, detail="No data returned from query1")
    try:
        # Log the column names
        logging.debug(f"Column names: {points[0].keys()}")
        # df1 = await delta_energy_dataframe(points)
        df1 = delta_energy_dataframe(points)
        logging.debug("Query_1 delta_energy_dataframe executed successfully")               
    except:
        print("Error processing query1")
        return None
        #raise HTTPException(status_code=400, detail="Error processing query1")    
    # Optional second query
    if str_query2 is not None:
        result = client.query(str_query2)
        logging.debug("Query_2 executed successfully")

        # Convert the result to a DataFrame
        points = list(result.get_points())
        if not points:
            # raise HTTPException(status_code=400, detail="No data returned from query2")
            logging.debug("Query_2 has not points")
            df = df1
            df.reset_index(inplace=True) 
        else:
            logging.debug("Query_2 has points")
            try:
                # df2 = await delta_energy_dataframe(points)
                df2 = delta_energy_dataframe(points)
                logging.info("Query_2 delta_energy_dataframe executed successfully")
                # Merge DataFrames
                df = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
                logging.debug("df = pd.merge(df1, df2, left_index=True, right_index=True, how='outer') OK")
                # The next line is replaced because Prophet can manage perfectly the NaN values
                # df['delta_energy'] = df['delta_energy_x'].fillna(0) + df['delta_energy_y'].fillna(0) # Sum the columns and fill NaN with the present values
                df['delta_energy'] = df['delta_energy_x'] + df['delta_energy_y'] # Sum the columns     
                df.reset_index(inplace=True)                 
                df = df[['time', 'delta_energy']]
                logging.debug("df = df[['time', 'delta_energy']] OK")
            except Exception as e:
                            logging.debug(f"Error processing dataframes in query2: {str(e)}")
                            return None
                            # raise HTTPException(status_code=400, detail=f"Error processing query2: {str(e)}")             
    else:
        df = df1
        df.reset_index(inplace=True)
    # Validate that the DataFrame is not empty
    if df is None or df.empty:
        print("No InfluxDB data returned")
        return None
        # raise HTTPException(status_code=400, detail="No InfluxDB data returned")
    
    # Validate that the DataFrame has exactly two columns
    if df.shape[1] != 2:
        print("Expected two columns 'time' and the query value")
        return None
        # raise HTTPException(status_code=400, detail="Expected two columns 'time' and the query value")

    try:
        # Assign names to the columns
        df.columns = ['ds', 'y']

        # Configure and train the Prophet model
        model = Prophet()
        model.fit(df)
        future = model.make_future_dataframe(periods=futurePeriods, freq=futureFreq)
        forecast = model.predict(future)
        forecast=forecast.tail(futurePeriods)

        # Convert dates to ISO format with timezone and create the output dictionary
        forecast['ds'] = pd.to_datetime(forecast['ds']).dt.tz_localize('UTC')
        forecast = forecast.set_index('ds')
        response = forecast.to_dict()['yhat'] 

        return response
    except Exception as e:
        logging.error(f"Error processing Prophet model: {e}")
        return None
        # raise HTTPException(status_code=500, detail=str(e))


# Probar las funciones que se llevarán al contenedor
El contenedor recibirá una petición en su puerto 5000 y FastAPI estará escuchando.
La función `post_energy_queries` simula que FastAPI recibe una llamada en el endpoint `energy_queries`

In [72]:
end=datetime.now()
entity_id1 = "victron_vebus_acin1toacout_228"
str_query1 = energy_query_string(entity_id1, end=end)
entity_id2 = "victron_vebus_invertertoacout_228"
str_query2 = energy_query_string(entity_id2, end=end)
energy_query_data = {
    "str_query1": str_query1,
    "str_query2": str_query2,
    "influx_host": "192.168.0.100",
    "influx_port": 8086,
    "influx_user": "homeassistant",
    "influx_password": password,
    "influx_dbname": "homeassistant",
    "futurePeriods": 12
}

response= post_energy_queries(str_query1, str_query2, futurePeriods=12, futureFreq='h')
response

DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\Enrique\AppData\Local\Temp\tmp66m2f9tf\y9ab472t.json
DEBUG:cmdstanpy:input tempfile: C:\Users\Enrique\AppData\Local\Temp\tmp66m2f9tf\n_0g2ua1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\vmware_share\\tfg_dev_env\\venv\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=46423', 'data', 'file=C:\\Users\\Enrique\\AppData\\Local\\Temp\\tmp66m2f9tf\\y9ab472t.json', 'init=C:\\Users\\Enrique\\AppData\\Local\\Temp\\tmp66m2f9tf\\n_0g2ua1.json', 'output', 'file=C:\\Users\\Enrique\\AppData\\Local\\Temp\\tmp66m2f9tf\\prophet_modelct47287q\\prophet_model-20241117184944.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:49:44 - cmdstanpy - INFO - Chain [1] star

{Timestamp('2024-11-17 17:00:00+0000', tz='UTC'): 0.1729716965708134,
 Timestamp('2024-11-17 18:00:00+0000', tz='UTC'): 0.1977675011804067,
 Timestamp('2024-11-17 19:00:00+0000', tz='UTC'): 0.20626089835347683,
 Timestamp('2024-11-17 20:00:00+0000', tz='UTC'): 0.19577024922440245,
 Timestamp('2024-11-17 21:00:00+0000', tz='UTC'): 0.18027562577661072,
 Timestamp('2024-11-17 22:00:00+0000', tz='UTC'): 0.17037818253478595,
 Timestamp('2024-11-17 23:00:00+0000', tz='UTC'): 0.16092178068454124,
 Timestamp('2024-11-18 00:00:00+0000', tz='UTC'): 0.13961356493049418,
 Timestamp('2024-11-18 01:00:00+0000', tz='UTC'): 0.10632599810922162,
 Timestamp('2024-11-18 02:00:00+0000', tz='UTC'): 0.08041872872977981,
 Timestamp('2024-11-18 03:00:00+0000', tz='UTC'): 0.0855692701214539,
 Timestamp('2024-11-18 04:00:00+0000', tz='UTC'): 0.1258945697293666}

## Simular como recibe esto el custom component

In [73]:
# Cuando el custom component reciba estos datos deberá convertirlos a la hora local definida en class_local_timezone 
cc_response = {str(k): v for k, v in response.items()}
# Convertir las claves a la hora local
cc_response = {pd.to_datetime(k).tz_convert(class_local_timezone).strftime('%Y-%m-%dT%H:%M:%S'): v for k, v in cc_response.items()}
cc_response

{'2024-11-17T18:00:00': 0.1729716965708134,
 '2024-11-17T19:00:00': 0.1977675011804067,
 '2024-11-17T20:00:00': 0.20626089835347683,
 '2024-11-17T21:00:00': 0.19577024922440245,
 '2024-11-17T22:00:00': 0.18027562577661072,
 '2024-11-17T23:00:00': 0.17037818253478595,
 '2024-11-18T00:00:00': 0.16092178068454124,
 '2024-11-18T01:00:00': 0.13961356493049418,
 '2024-11-18T02:00:00': 0.10632599810922162,
 '2024-11-18T03:00:00': 0.08041872872977981,
 '2024-11-18T04:00:00': 0.0855692701214539,
 '2024-11-18T05:00:00': 0.1258945697293666}

# Instalar el Addon y probarlo
Una vez desarrollado el Addon en instalado en la maquina de producción vamos a hacer algunas pruebas


In [74]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import pytz

base_url = "http://192.168.0.100:5000" # Raspeberry Pi (HA OS) runs the container

## Endpoint `/forecast`
Este endpoint sirve para poder enviar datos generados fuera de InfluxDB y utilizar Prophet para realizar predicciones

In [75]:
forecast_data = {
    "data": [
        {"ds": "2023-01-01", "y": 10},
        {"ds": "2023-01-02", "y": 15},
        {"ds": "2023-01-03", "y": 20}
    ],
    "futurePeriods": 4,
    "futureFreq": "d"
}



# Send POST request to the /forecast endpoint
response = requests.post(f"{base_url}/forecast", json=forecast_data)
print("Forecast response:")
response.json()

Forecast response:


{'2023-01-04T00:00:00': 25.000000000000004,
 '2023-01-05T00:00:00': 30.000000000000004,
 '2023-01-06T00:00:00': 35.00000000000001,
 '2023-01-07T00:00:00': 40.00000000000001}

## Endpoint `/query`
Este endpoint sirve para enviar consultas a InfluxDB en formato InfluxQL que entrenaran el modelo para obtener las predicciones. 
Observar que la primera hora del pronostico es la hora siguiente al endtime. Se destaca aquí este hecho ya que en el endpoint para consultas de energía el funcionamiento es diferente ya que la primera hora de la predicción es siempre la hora actual.
Este comportamiento es fácilmente modificable por el usuario del endpoint `query` estableciendo el endtime justo en el periodo anterior, de forma que su utilización es lo más versátil posible.


In [76]:
end=datetime.now() # La primera predicción será para la hora siguiente
#end=datetime.now()-timedelta(hours=1) # La primera predicción será para la hora actual
start=end-timedelta(days=10)
entity_id1 = "energy_current_hour"
str_query = energy_query_string(entity_id1, start=start, end=end)
# El parámetro futureFreq es opcional y por defecto es 'h'. Se puede cambiar a 'd' para predecir días.
# En el caso del custom component siempre estaremos interesados en valores por horas.
query_data = {
    "str_query": str_query,
    "futurePeriods": 10,
    "futureFreq": "h"
}

# Send request POST endpoint /query
response = requests.post(f"{base_url}/query", json=query_data)
if response.status_code == 200:
    print("Query response:", response.json())
    cc_response = {str(k): v for k, v in response.json().items()}
    cc_response = {pd.to_datetime(k).tz_convert(class_local_timezone)\
                    .strftime('%Y-%m-%dT%H:%M:%S'): v for k, v in cc_response.items()}
    print()
    print("\nQuery response local timezone:")        
else:
    print(f"Request failed with status code {response.status_code}")
cc_response

Query response: {'2024-11-17T18:00:00+00:00': 0.03516269139561684, '2024-11-17T19:00:00+00:00': -0.022256173162756743, '2024-11-17T20:00:00+00:00': -0.008285875749996674, '2024-11-17T21:00:00+00:00': 0.02793003768781746, '2024-11-17T22:00:00+00:00': 0.043981223480698106, '2024-11-17T23:00:00+00:00': 0.03155439412627109, '2024-11-18T00:00:00+00:00': 0.011679798647356299, '2024-11-18T01:00:00+00:00': 0.007322526311173971, '2024-11-18T02:00:00+00:00': 0.02073322483218748, '2024-11-18T03:00:00+00:00': 0.03478725110497144}


Query response local timezone:


{'2024-11-17T19:00:00': 0.03516269139561684,
 '2024-11-17T20:00:00': -0.022256173162756743,
 '2024-11-17T21:00:00': -0.008285875749996674,
 '2024-11-17T22:00:00': 0.02793003768781746,
 '2024-11-17T23:00:00': 0.043981223480698106,
 '2024-11-18T00:00:00': 0.03155439412627109,
 '2024-11-18T01:00:00': 0.011679798647356299,
 '2024-11-18T02:00:00': 0.007322526311173971,
 '2024-11-18T03:00:00': 0.02073322483218748,
 '2024-11-18T04:00:00': 0.03478725110497144}

## Endpoint `/energy_queries`
Este endpoint sirve para enviar consultas a InfluxDB en formato InfluxQL que entrenaran el modelo para obtener las predicciones. Está especialmente pensado para valores de energia que se registran de forma acumulada al estilo de un contador tradicional. Dado que el interés de cara al custom component es conocer la energía que se consumira durante periodos de una hora, se ha creado esta funcionalidad particular. Ademas permite la definición de dos consultas por si los valores de interes estuviesen en dos sensores diferentes como sucede en el caso de los inversores Victron, donde no existe un sensor de energía (sí de potencia) que mida la energía total consumida y es por tanto es necesario sumar el valor de dos sensores. 
Observar que la primera hora del pronostico es la hora correspondiente al endtime. Al contrario de como se definió en el endpoint  `query` se ha considerado más oportuno trabajar solo con datos consolidados de manera predeterminada. 
En principio la intención es que el parámetro start sea siempre None para que el modelo se entrene con todos los datos disponibles en InfluxDB.

In [77]:
end=datetime.now()
start=end-timedelta(days=10)
entity_id1 = "victron_vebus_acin1toacout_228"
str_query1 = energy_query_string(entity_id1, start=None, end=end)
entity_id2 = "victron_vebus_invertertoacout_228"
str_query2 = energy_query_string(entity_id2, start=None, end=end)
# El parámetro str_query2 es opcional. Si no se especifica se devolverá solo la predicción tomando los datos para el entrenamiento de str_query1
energy_query_data = {
    "str_query1": str_query1,
    "str_query2": str_query2,
    "futurePeriods": 12,
    "futureFreq": "h"
}

# Send Request POST endpoint /energy_queries
response = requests.post(f"{base_url}/energy_queries", json=energy_query_data)
if response.status_code == 200:
    print("Energy Queries response:", response.json())
    cc_response = {str(k): v for k, v in response.json().items()}
    cc_response = {pd.to_datetime(k).tz_convert(class_local_timezone)\
                    .strftime('%Y-%m-%dT%H:%M:%S'): v for k, v in cc_response.items()}
    print("\nEnergy Queries response local timezone:")
else:
    print(f"Request failed with status code {response.status_code}")
cc_response

Energy Queries response: {'2024-11-17T17:00:00+00:00': 0.17296849427060565, '2024-11-17T18:00:00+00:00': 0.1977641829382623, '2024-11-17T19:00:00+00:00': 0.2062575933838498, '2024-11-17T20:00:00+00:00': 0.19576703840023957, '2024-11-17T21:00:00+00:00': 0.1802724766165519, '2024-11-17T22:00:00+00:00': 0.17037500480074547, '2024-11-17T23:00:00+00:00': 0.16091854147678145, '2024-11-18T00:00:00+00:00': 0.1396103416656993, '2024-11-18T01:00:00+00:00': 0.1063229106262815, '2024-11-18T02:00:00+00:00': 0.0804158149687874, '2024-11-18T03:00:00+00:00': 0.08556642957528197, '2024-11-18T04:00:00+00:00': 0.1258916345598044}

Energy Queries response local timezone:


{'2024-11-17T18:00:00': 0.17296849427060565,
 '2024-11-17T19:00:00': 0.1977641829382623,
 '2024-11-17T20:00:00': 0.2062575933838498,
 '2024-11-17T21:00:00': 0.19576703840023957,
 '2024-11-17T22:00:00': 0.1802724766165519,
 '2024-11-17T23:00:00': 0.17037500480074547,
 '2024-11-18T00:00:00': 0.16091854147678145,
 '2024-11-18T01:00:00': 0.1396103416656993,
 '2024-11-18T02:00:00': 0.1063229106262815,
 '2024-11-18T03:00:00': 0.0804158149687874,
 '2024-11-18T04:00:00': 0.08556642957528197,
 '2024-11-18T05:00:00': 0.1258916345598044}